In [4]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import matplotlib
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers
from datetime import datetime
import quantstats as qstats
%matplotlib inline

# Read the CSV file using pandas
df = pd.read_csv('prices.csv')

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

stock_start_date = df['date'].min() # Earliest Start Date in all Stock Data Start Dates
stock_end_date = df['date'].max() # Latest End Date in all Stock Data Start Dates

starting_dict = {} # To hold Start Dates for all 25 stocks

# Creating a DataFrame with all combinations of tickers and dates
tickers = df['ticker'].unique()
dates = pd.date_range(start=stock_start_date, end=stock_end_date, freq='D')
index = pd.MultiIndex.from_product([tickers, dates], names=['ticker', 'date'])
dummy_df = pd.DataFrame(index=index).reset_index()

# Populating the starting_dict dictionary with the starting date of each stock
for ticker in tickers:
    starting_dict[ticker] = df[df['ticker'] == ticker]['date'].min()

# Adding 200 days to the starting date of each stock
for ticker in tickers:
    starting_dict[ticker] = starting_dict[ticker] + pd.Timedelta(days=200)

# Merging the dummy DataFrame with the original DataFrame on the 'ticker' and 'date' columns
merged_df = pd.merge(dummy_df, df, on=['ticker', 'date'], how='left')

# Backward filling missing values in the 'close' column with the next available value within each group of 'ticker'
merged_df[['close','open','high','low']] = merged_df.groupby('ticker')[['close','open','high','low']].fillna(method='bfill')

# Updating the original DataFrame with the merged DataFrame
df = merged_df

# Converting the 'date' column to a datetime object with the specified format again (this line is redundant)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Setting the 'date' column as the index of the DataFrame
df = df.set_index('date')

df

class MyStrategy(bt.Strategy):

    params = (
        ('sma_period', 200),  # SMA period
        ('lookback_period', 200),  # Lookback Period to calculate returns
        ('rebalance_day', 1)  # Rebalancing at the Start of Each Month
    )

    def __init__(self):
        self.first_trading_day = False
        self.current_month = None
        self.prev_month = None
        self.has_position = False
        self.start_date = starting_dict
        self.sma_200 = [bt.ind.SMA(data, period=self.params.sma_period) for data in self.datas]
        self.selected_stocks = []
        self.selected_data = []
        self.rebalance_dates = set()  # Store rebalance dates

    def filter_stocks(self):
        self.selected_data = []
        for i, d in enumerate(self.datas):
            if pd.Timestamp(d.datetime.date(0)) > self.start_date[d._name]:
                if self.datas[i].close[0] > self.sma_200[i][0]:
                    self.selected_data.append(d._name)

                elif self.datas[i].close[0] < self.sma_200[i][0]:
                    pass

        self.selected_stocks = [d for d in self.datas if d._name in self.selected_data]        


        if len(self.selected_stocks)>0:
            self.get_returns(self.selected_stocks)
            
        for stock in self.selected_stocks:
            first_day_of_month = stock.datetime.date().replace(day=1)
            self.rebalance_dates.add(first_day_of_month)

    def next(self):
        # Check if it's a rebalance day
        if self.data.datetime.date().day == self.params.rebalance_day:
            if self.data.datetime.date() in self.rebalance_dates:
                if self.selected_stocks:
                    print(f"\nSelected Stocks for {self.data.datetime.date()}:")
                    for stock in self.selected_stocks:
                        print(stock._name)

    def mean_variance_optimization(self,returns):
        n = returns.shape[1]
        Sigma = np.cov(returns.T)
        if Sigma.shape == ():
            Sigma = np.array([[Sigma]])
        P = matrix(Sigma)
        q = matrix(np.zeros((n, 1)))
        G = matrix(-np.eye(n))
        h = matrix(np.zeros((n, 1)))
        A = matrix(np.ones((1, n)))
        b = matrix(np.array([1.]))
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        return np.array(sol['x']).flatten()
    

    def get_returns(self, selected_stocks):
        prices = np.zeros((self.params.lookback_period,len(selected_stocks)))
        returns = np.zeros((self.params.lookback_period,len(selected_stocks)))

        for i, d in enumerate(selected_stocks):
            prices[:, i] = d.close.get(size=self.params.lookback_period)
            returns[1:, i] = np.diff(prices[:, i]) / prices[:-1, i]

        weights = self.mean_variance_optimization(returns)

        self.place_order(weights, selected_stocks)

    def place_order(self, weights, selected_stocks):
    # Selling the stocks not selected
        for i, d in enumerate(self.datas):
            if d not in selected_stocks:
                self.order_target_percent(data=d, target=0.0)

        # Buying the selected ones
        date_printed = False  # Flag to track if date has been printed
        for i, d in enumerate(selected_stocks):
            stock_name = d._name
            weight = weights[i]
            self.order_target_percent(data=d, target=weight)

            # Display date if not printed yet
            if not date_printed:
                print(f"Date - {self.data.datetime.date()}")
                date_printed = True

            # Print stock name and weight
            print(f"{stock_name} - {weight:.2f}")
            
        print()  # Print a new line after each block of date
          
    def next(self):
        self.current_month = self.data.datetime.date().month
        if self.prev_month is None or self.current_month != self.prev_month:
            self.first_trading_day = True
        else:
            self.first_trading_day = False
        if self.first_trading_day:
            self.filter_stocks()

        self.prev_month = self.current_month


# Create an empty cerebro instance
cerebro = bt.Cerebro()

# Add your strategy to cerebro
cerebro.addstrategy(MyStrategy)

# Looping over each unique value in the 'ticker' column of the DataFrame
for ticker in df['ticker'].unique():
    # Creating a new DataFrame with only rows where the 'ticker' column is equal to the current ticker
    stock_df = df[df['ticker'] == ticker]
    
    # Creating a data feed from the new DataFrame using the 'bt.feeds.PandasData' class
    datafeed = bt.feeds.PandasData(dataname=stock_df,
                                  open='close',
                                  close='close',
                                  high='close',
                                  low='close',
                                  volume=None,
                                  openinterest=None,
                                  datetime=None,
                                  fromdate=datetime(2010, 1, 4),
                                  todate=datetime(2020, 6, 15))
    
    # Adding the data feed to the Cerebro instance with a name equal to the current ticker
    cerebro.adddata(datafeed, name=ticker)

# Set the cash for cerebro
cerebro.broker.setcash(100000.00)

# Adding the PyFolio analyzer to the Cerebro instance
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Run cerebro
strategy_run = cerebro.run()

print('Final Portfolio Value:', cerebro.broker.getvalue())

# Set the figure size for the plot
plt.rcParams['figure.figsize'] = [10,50]

# Update the font size for the plot
plt.rcParams.update({'font.size': 12})

# Analyze results and plot the backtest
cerebro.plot(volume=False)

# Get the final strategy object
strategy_final = strategy_run[-1]

# Get the pyfolio analyzer from the strategy
pyfolio_object = strategy_final.analyzers.getbyname('pyfolio')

# Retrieve various portfolio items from the pyfolio analyzer
stock_returns, positions, transactions, gross_lev = pyfolio_object.get_pf_items()

# Remove timezone information from the index of stock_returns
stock_returns.index = stock_returns.index.tz_convert(None)

# Generate full performance report using quantstats library
try:
    qstats.reports.full(stock_returns)
except TypeError:
    pass

plt.savefig('quant_stats_final.png')

import quantstats as qs

# Extract the PyFolio analyzer results from the strategy run
pyfolio_analyzer = strategy_run[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfolio_analyzer.get_pf_items()

# Convert the returns series to a DataFrame and reformat the index
returns_df = pd.DataFrame(returns)
returns_df.index = returns_df.index.tz_convert(None)

# Use QuantStats to generate a report and plot the results
qs.reports.html(returns_df['return'], output='report.html')
qs.plots.snapshot(returns_df['return'], title='Strategy Performance')



Date - 2010-08-01
AXISBANK - 0.00
BAJAJFINSV - 0.00
BAJAJHLDNG - 0.17
BAJFINANCE - 0.00
BANKBARODA - 0.10
HDFC - 0.08
HDFCBANK - 0.20
ICICIBANK - 0.00
INDUSINDBK - 0.00
KOTAKBANK - 0.00
PEL - 0.02
PFC - 0.13
PNB - 0.18
SBIN - 0.00
SRTRANSFIN - 0.11

Date - 2010-09-01
AXISBANK - 0.00
BAJAJFINSV - 0.00
BAJAJHLDNG - 0.13
BAJFINANCE - 0.00
BANKBARODA - 0.09
HDFC - 0.07
HDFCBANK - 0.22
ICICIBANK - 0.00
INDUSINDBK - 0.00
KOTAKBANK - 0.02
PEL - 0.03
PFC - 0.15
PNB - 0.18
SBIN - 0.00
SRTRANSFIN - 0.10

Date - 2010-10-01
AXISBANK - 0.00
BAJAJFINSV - 0.01
BAJAJHLDNG - 0.11
BAJFINANCE - 0.01
BANKBARODA - 0.12
HDFC - 0.05
HDFCBANK - 0.22
ICICIBANK - 0.00
INDUSINDBK - 0.01
KOTAKBANK - 0.01
PEL - 0.05
PFC - 0.12
PNB - 0.18
SBIN - 0.00
SRTRANSFIN - 0.10

Date - 2010-11-01
AXISBANK - 0.00
BAJAJFINSV - 0.02
BAJAJHLDNG - 0.09
BAJFINANCE - 0.00
BANKBARODA - 0.14
HDFC - 0.03
HDFCBANK - 0.23
ICICIBANK - 0.00
INDUSINDBK - 0.03
KOTAKBANK - 0.01
PFC - 0.11
PNB - 0.20
SBIN - 0.00
SRTRANSFIN - 0.14

Date - 2010

<IPython.core.display.Javascript object>

                           Strategy
-------------------------  ----------
Start Period               2010-01-04
End Period                 2020-06-15
Risk-Free Rate             0.0%
Time in Market             64.0%

Cumulative Return          253.61%
CAGR﹪                     12.84%

Sharpe                     0.51
Prob. Sharpe Ratio         99.1%
Smart Sharpe               0.51
Sortino                    0.89
Smart Sortino              0.89
Sortino/√2                 0.63
Smart Sortino/√2           0.63
Omega                      1.17

Max Drawdown               -49.13%
Longest DD Days            606
Volatility (ann.)          19.86%
Calmar                     0.26
Skew                       13.79
Kurtosis                   584.63

Expected Daily %           0.03%
Expected Monthly %         1.01%
Expected Yearly %          12.17%
Kelly Criterion            7.77%
Risk of Ruin               0.0%
Daily Value-at-Risk        -2.02%
Expected Shortfall (cVaR)  -2.02%

Max Consecutive Wins   

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2019-10-31,2020-05-23,2020-06-15,228,-49.126128,-48.716424
2,2013-05-31,2013-12-17,2014-07-24,419,-25.222211,-24.519905
3,2010-11-05,2011-10-05,2012-07-03,606,-22.682982,-21.031479
4,2016-09-24,2016-12-24,2017-03-30,187,-20.100408,-18.742708
5,2018-07-28,2018-10-09,2019-05-01,277,-16.781503,-15.378030


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>